
---



# **SiameseNet for Recognizing Faces in the Wild**

---



In [0]:
from google.colab import drive
drive.mount('/content/drive')

!pip install git+https://github.com/rcmalli/keras-vggface.git

!unzip drive/My\ Drive/Proyecto/data.zip -d content/
!unzip drive/My\ Drive/Proyecto/test.zip -d content/test/
!unzip drive/My\ Drive/Proyecto/sample_submission.zip -d content/

# **Imports:**

In [0]:
import glob
import os
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from keras.layers import Input, Dense, Dropout, Lambda
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras import regularizers
from keras.utils.vis_utils import plot_model
from IPython.display import display

# **Read Data:**

In [0]:
def read_image(path):
    """
    Funcion que permite leer imagenes a partir de un archivo
    Args:
        path: Ruta de la imagen
    """
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)


def read_family_members_images(data_path):
    """
    Funcion que procesa la ruta especificada como parametro. Obtiene una lista
    con los miembros de cada familia, la cual tendra el formato "FXXXX/MIDY",
    y un diccionario con las rutas de las imagenes de cada miembro de cada familia.

    Args:
        data_path: Ruta de los archivos a procesar.
    
    Return:
        Devuelve una lista con los miembros de cada familia y un diccionario con
        las imagenes de cada miembro de cada familia.
    """
    # Leer la ruta proporcionada y obtener todos los directorios
    # Cada directorio esta asociado a una familia
    dirs = sorted(list(glob.glob(data_path + "*")))

    # Obtener los nombres de los directorios de las familias
    family_dirs = np.array([dir.split("/")[-1] for dir in dirs])

    # Obtener imagenes asociadas a cada directorio
    images = {f"{family}/{member.split('/')[-1]}": sorted(list(glob.glob(member + "/*.jpg")))
        for family in family_dirs for member in sorted(list(glob.glob(f"{data_path}{family}/*")))
    }
    
    family_members_list = list(images.keys())

    return family_members_list, images



# **Data Generators:**

In [0]:
def generate_datasets(families, val_prop=0.2):
    """
    Funcion que permite generar los datasets de train, test y validacion
    a partir de un array de directorios, los cuales representan las familias.
    Los datos son mezclados para que se escoja de forma aleatoria.

    Args:
        families: Array con los directorios de las familias.
        test_prop: Proporcion de los datos totales que tienen que estar en el
                   conjunto de test.
        val_prop: Proporcion de los (datos_totales - datos_test) que tienen que
                  estar en el conjunto de validacion.
    
    Return:
        Devuelve un array con los directorios de las familias que forman el conjunto
        de train, otro para el conjunto de validacion y otro para el conjunto de test.
    """
    # Mezclar familias
    shuffle_families = np.copy(families)
    np.random.shuffle(shuffle_families)

    # Obtener la ultima proporcion de las familias y guardarla en el conjunto
    # de test
    idx_val = int(len(shuffle_families) * (1 - val_prop))
    val_dirs = shuffle_families[idx_val:]
    train_dirs = shuffle_families[:idx_val]

    # Volver a mezclar familias para escoger conjunto de validacion
    np.random.shuffle(train_dirs)

    print(train_dirs.shape)
    print(val_dirs.shape)

    return train_dirs, val_dirs



def dataset_to_images(dataset, images, relationships, size, relationships_prop):
    """
    Funcion que genera dos arrays de individuos con un tamaño determinado, y otro que
    nos indica el parentesco entre un par de individuos de cada uno de los arrays anteriores.
    Los datos se escogeran de forma aleatoria entre todos los individuos proporcionados.
    Args:
        dataset: Array con los directorios de las familias.
        images: Array con los directorios de las imagenes de cada individuo de la familia.
        relationships: Relaciones entre los individuos a procesar.
        size: Tamaño de los datos a generar.
        relationships_prop: Proporcion de individuos con un parentesco familiar que tendran
                            los datos generados.
    
    Return:
        Devuelve dos arrays con los individuos que serán procesados por cada una de las
        partes de nuestra red, y un array con los parentescos entre los arrays anteriores
    """    
    left_images = []
    right_images = []
    targets = []

    # Elegir los 1's
    while len(left_images) < int(size*relationships_prop):
        # Escogemos una linea aleatoria del CSV
        index = np.random.choice(len(relationships))
        ind = relationships[index]

        # Comprobamos que los individuos estan en el dataset
        if ind[0] in dataset and ind[1] in dataset:
            # Elegimos aleatoriamente una imagen de esos individuos
            left_images.append(read_image(np.random.choice( images[ind[0]] )))
            right_images.append(read_image(np.random.choice( images[ind[1]] )))
            targets.append(1.)

    # Elegir los 0's
    while len(left_images) < int(size):
        # Accedemos dos individuos diferentes aleatorios del dataset
        ind = np.random.choice(dataset, 2, replace=False)

        # Comprobamos si son parientes
        if (ind[0],ind[1]) not in relationships and (ind[1],ind[0]) not in relationships:
            # En caso afirmativo añadimos con etiqueta 1
            left_images.append( read_image( np.random.choice(images[ind[0]]) ) )
            right_images.append( read_image( np.random.choice(images[ind[1]]) ) )
            targets.append(0.0)
    
    left_images = np.array(left_images)
    right_images = np.array(right_images)
    targets = np.array(targets)

    idx_perm = np.random.permutation(size)

    left_images = left_images[idx_perm]
    right_images = right_images[idx_perm]
    targets = targets[idx_perm]

    return left_images, right_images, targets



def batch_generator(dataset, images, relationships_path, batch_size=32, relationships_prop=0.2):
    """
    Funcion que selecciona aleatoriamente dos conjunto de individuos
    y sus etiquetas, asigna una proporcion de parejas con parentesco
    entre los dos conjuntos, y devuelve en cada iteracion la
    cantidad asignada como tamaño de batch.
    Args:
        dataset: Array con los directorios de las familias.
        images: Array con los directorios de las imagenes de cada individuo
                de la familia.
        relationships_path: Ruta del archivo de relaciones a procesar.
        batch_size: Tamaño del batch.
        relationships_prop: Proporcion de individuos con un parentesco familiar
                            que tendran los datos generados.
    
    Return:
        Devuelve dos arrays con los individuos que serán procesados por cada una
        de las partes de nuestra red, y un array con los parentescos entre los
        arrays anteriores
    """
    # Leemos el archivo donde se encuentran las relaciones familiares entre individuos
    relationships = pd.read_csv(relationships_path)
    relationships = list(zip(relationships.p1.values, relationships.p2.values))
    

    while True:
        # Generamos un conjunto de imagenes aleatorias y lo devolvemos hasta
        # que el iterador vuelva a pedir otro
        left_images, right_images, targets = dataset_to_images(dataset, images, relationships, batch_size, relationships_prop)

        yield [left_images, right_images], targets

# **Graphics Generator:**

In [0]:
def show_history(hist):
    """
    Funcion que muestra dos graficas con la evolucion de la funcion de perdida
    y de la precision, respectivamente, obtenidas al entrenar nuestro modelo

    Args:
        hist: Registro de valores de entrenamiento y de validacion (si
              corresponde) en épocas sucesivas.
    """
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()

    acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['Training accuracy', 'Validation accuracy'])
    plt.show()


def visualize_model(model):
    """
    Funcion que muestra la arquitectura de un modelo

    Args:
        model: Modelo a mostrar
    """
    img = plot_model(model, show_shapes=True, show_layer_names=True)
    display(img)

# **Initializers implementation:**

In [0]:
def initialize_bias(shape, dtype=None):
    """
    The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)
  
def initialize_weights(shape, dtype=None):
    """
    The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

# **Binary Focal Loss implementation:**

In [0]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.

      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)

      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.

    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)

    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

# **SiameseNet implementation:**



In [0]:
def create_siamesenet(model, optimizer, loss_function):
    """
    Funcion que crea una red siamesa a partir de una serie de redes preentrenada
    con el conjunto de datos VGGFace2. Este modelo declara dos entradas y,
    posteriormente se conectan cada una de las partes mediante la distancia L1.
    
    Args:
        model: Indica cual de las tres redes ya entrenadas con VGGFace2
               utilizaremos en nuestro modelo: VGG16, RESNET50 o SENET50
        optimizer: Indica cual sera el optimizador de nuestro modelo. Se puede
                   crear una instancia antes de pasarlo, o puede llamarlo
                   directamente por su nombre.
        loss_function: Indica la funcion de perdida de nuestro modelo. Se puede
                       pasar el nombre de una funcion de perdida existente o
                       pasar una funcion simbolica.
    
    Return:
        Devuelve la red ya creada y muestra un resumen de esta
    """
    # Dimension de los datos de entrada
    shape = (224, 224, 3)

    # Declaramos 2 entradas, una para cada imagen
    left_input = Input(shape)
    right_input = Input(shape)

    # Generamos nuestro modelo entrenado con VGGFace
    vgg_model = VGGFace(model=model, include_top=False, weights="vggface", pooling="max")

    # Conectar las partes de la red a cada una de las entradas
    encoded_l = vgg_model(left_input)
    encoded_r = vgg_model(right_input)

    # Obtenemos la distancia L1 entre los dos tensores
    L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

    # Aniadimos la funcion de distancia y la última capa sigmoidal a la red
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1, activation='sigmoid')(L1_distance)

    # Creamos el modelo
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    # Compilamos con un optimizador y una funcion de perdida determinadas
    siamese_net.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    # Mostramos un resumen de la red
    siamese_net.summary()

    return siamese_net


# **Dense SiameseNet implementation:**

In [0]:
def create_dense_siamesenet(model, optimizer, loss_function):
    """
    Funcion que crea una red siamesa a partir de una serie de redes preentrenada
    con el conjunto de datos VGGFace2. Este modelo declara dos entradas y,
    posteriormente se conectan cada una de las partes mediante la distancia L1.
    Finalmente se la aniaden 2 capas FC para aplicar con los pesos obtenidos.
    
    Args:
        model: Indica cual de las tres redes ya entrenadas con VGGFace2
               utilizaremos en nuestro modelo: VGG16, RESNET50 o SENET50
        optimizer: Indica cual sera el optimizador de nuestro modelo. Se puede
                   crear una instancia antes de pasarlo, o puede llamarlo
                   directamente por su nombre.
        loss_function: Indica la funcion de perdida de nuestro modelo. Se puede
                       pasar el nombre de una funcion de perdida existente o
                       pasar una funcion simbolica.
    
    Return:
        Devuelve la red ya creada y muestra un resumen de esta
    """
    # Dimension de los datos de entrada
    shape = (224, 224, 3)

    # Declaramos 2 entradas, una para cada imagen
    left_input = Input(shape)
    right_input = Input(shape)

    # Generamos nuestro modelo entrenado con VGGFace
    vgg_model = VGGFace(model=model, include_top=False, weights="vggface", pooling="max")

    # Conectar las partes de la red a cada una de las entradas
    encoded_l = vgg_model(left_input)
    encoded_r = vgg_model(right_input)

    # Obtenemos la distancia L1 entre los dos tensores
    L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

    # Aniadimos la funcion de distancia
    L1_distance = L1_layer([encoded_l, encoded_r])

    # Aniadimos 2 capas FC y otras 2 de Dropout tras cada una
    x = Dense(100, activation="relu")(L1_distance)
    x = Dropout(0.1)(x)
    x = Dense(25, activation="relu")(x)
    x = Dropout(0.1)(x)

    # Aniadimos la ultima capa sigmoidal a la red
    prediction = Dense(1, activation='sigmoid')(x)

    # Creamos el modelo
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    # Compilamos con un optimizador y una funcion de perdida determinadas
    siamese_net.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    # Mostramos un resumen de la red
    siamese_net.summary()

    return siamese_net


# **Regularized SiameseNet implementation:**

In [0]:
def create_regularized_siamesenet(model, optimizer, loss_function):
    """
    Funcion que crea una red siamesa a partir de una serie de redes preentrenada
    con el conjunto de datos VGGFace2. Este modelo declara dos entradas y,
    posteriormente se conectan cada una de las partes mediante la distancia L1.
    Finalmente se la aniaden 2 capas FC inicializando los pesos y el bias, y
    con una funcion de regularizacion l2.
    
    Args:
        model: Indica cual de las tres redes ya entrenadas con VGGFace2
               utilizaremos en nuestro modelo: VGG16, RESNET50 o SENET50
        optimizer: Indica cual sera el optimizador de nuestro modelo. Se puede
                   crear una instancia antes de pasarlo, o puede llamarlo
                   directamente por su nombre.
        loss_function: Indica la funcion de perdida de nuestro modelo. Se puede
                       pasar el nombre de una funcion de perdida existente o
                       pasar una funcion simbolica.
    
    Return:
        Devuelve la red ya creada y muestra un resumen de esta
    """
    # Dimension de los datos de entrada
    shape = (224, 224, 3)

    # Declaramos 2 entradas, una para cada imagen
    left_input = Input(shape)
    right_input = Input(shape)

    # Generamos nuestro modelo entrenado con VGGFace
    vgg_model = VGGFace(model=model, include_top=False, weights="vggface", pooling="max")

    # Conectar las partes de la red a cada una de las entradas
    encoded_l = vgg_model(left_input)
    encoded_r = vgg_model(right_input)

    # Obtenemos la distancia L1 entre los dos tensores
    L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

    # Aniadimos la funcion de distancia
    L1_distance = L1_layer([encoded_l, encoded_r])

    # Aniadimos 2 capas FC inicializadas y regularizadas,
    # y otras 2 de Dropout tras cada una
    x = Dense(100, activation="relu", kernel_regularizer=regularizers.l2(1e-3),
                                      kernel_initializer=initialize_weights,
                                      bias_initializer=initialize_bias)(L1_distance)
    x = Dropout(0.1)(x)
    x = Dense(25, activation="relu", kernel_regularizer=regularizers.l2(1e-3),
                                     kernel_initializer=initialize_weights,
                                     bias_initializer=initialize_bias)(x)
    x = Dropout(0.1)(x)

    # Aniadimos la ultima capa sigmoidal a la red
    prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(x)

    # Creamos el modelo
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    # Compilamos con un optimizador y una funcion de perdida determinadas
    siamese_net.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    # Mostramos un resumen de la red
    siamese_net.summary()

    return siamese_net


# **Main**

## Reading and preprocessing data


In [0]:
# Establecer semilla
np.random.seed(1)

# Establecer si se van a entrenar todas las redes
train_all_nets = False

# Lectura y preprocesado de los datos
train_folders_path = "content/train/"
train_relationships = "content/train_relationships.csv"

dirs, images = read_family_members_images(train_folders_path)
train_dirs, val_dirs = generate_datasets(dirs)
print(train_dirs)

## Train All Tested Networks

In [0]:
# Entrenar todas las redes probadas si se ha especificado
if train_all_nets:
    # Establecer optimizadores que se utilizaran
    adam = Adam()
    adam_mod = Adam(0.00001)
    rmsprop = RMSprop()

    # Establecer todas las funciones de perdida que se utilizaran
    binary_loss = "binary_crossentropy"
    focal_loss = [binary_focal_loss(alpha=.25, gamma=2)]

    # Crear todos los modelos probados (menos el definitivo)
    tested_models = [
        create_siamesenet('vgg16', adam, binary_loss),
        create_siamesenet('resnet50', adam, binary_loss),
        create_siamesenet('senet50', adam, binary_loss),
        create_siamesenet('resnet50', adam_mod, binary_loss),
        create_dense_siamesenet('resnet50', adam_mod, binary_loss),
        create_dense_siamesenet('resnet50', rmsprop, binary_loss),
        create_dense_siamesenet('resnet50', adam_mod, focal_loss)
    ]

    # Entrenar cada uno de los modelos y mostrar historia
    for model in tested_models:
        hist = model.fit_generator(batch_generator(train_dirs, images, train_relationships, batch_size=32, relationships_prop=0.6),
                    validation_data=batch_generator(val_dirs, images, train_relationships, batch_size=32, relationships_prop=0.6),
                    epochs=30, verbose=1, steps_per_epoch=100, validation_steps=30)

        show_history(hist)

## Creating Network

In [0]:
# Instanciamos optimizador y funcion de perdida
optimizer = Adam(0.00001)
loss_function = "binary_crossentropy"

# Creamos la red
model = create_regularized_siamesenet('resnet50', optimizer, loss_function)

# Imprimimos la arquitectura de la red
visualize_model(model)

## Fitting Network

In [0]:
hist = model.fit_generator(batch_generator(train_dirs, images, train_relationships, batch_size=32, relationships_prop=0.6),
                    validation_data=batch_generator(val_dirs, images, train_relationships, batch_size=32, relationships_prop=0.6),
                    epochs=30, verbose=1, steps_per_epoch=100, validation_steps=30)

show_history(hist)

# **Test Data Generator:**

In [0]:
def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

from tqdm import tqdm

test_path = "content/test/"
submission = pd.read_csv("content/sample_submission.csv")

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_image(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_image(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions
submission.to_csv("drive/My Drive/Proyecto/siamese_net.csv", index=False)
print(submission)